<b>Назначение:</b> детекция/выравнивание/вырезание лиц

In [3]:
from PIL import Image
import pandas as pd
import torch
from tqdm import tqdm
from collections import Counter
import numpy as np
import math
from ultralytics import YOLO
import cv2

IMAGES_INFO_TABLE = '../data/images_info.csv'
CROPED_IMAGES_DIR = 'data/croped_images'
IMAGE_SIZE = 64

In [4]:
class ImageProcessing:
    def __init__(self) -> None:
        self.face_detector = YOLO('../yolov8n-face.pt')

    #
    def get_central_of_face(self, face_info):
        left_eye, right_eye, nose = face_info['key_points'][:3]
        center_x = (left_eye[0] + right_eye[0] + nose[0]) // 3
        center_y = (left_eye[1] + right_eye[1] + nose[1]) // 3
        face_center_point = [center_x, center_y]
        return face_center_point

    #
    def detect_faces(self, image):
        res = self.face_detector(image, show=False, save=False, conf=0.4,
                                 save_txt=False, save_crop=False, verbose=False)[0]
        
        faces = []
        for ind in range(len(res.boxes)):
            box_points = res.boxes.xyxy[ind].cpu().numpy().astype(int)
            keypoints = res.keypoints[ind].data.cpu().numpy()[0][:, :2].astype(int)
            tmp_face = {
                "conf": res.boxes.conf[ind].cpu().item(),
                "box_points": box_points.tolist(),
                "key_points": keypoints.tolist(),
                }
            
            f_center = self.get_central_of_face(tmp_face)
            tmp_face['face_center'] = f_center
            faces.append(tmp_face)

        return faces

    #
    def get_modif_image(self, image):
        faces_info = self.detect_faces(image)

        img_copy = image.copy()
        draw = PIL.ImageDraw.Draw(img_copy)
        p_wdth = 4

        for face in faces_info:
            box, kp = face['box_points'], face['key_points']
            
            draw.rectangle(box, width=2, outline='red')
            
            points = kp + [face['face_center']]

            colors = ['red', 'blue', 'green', 'orange', 'black', 'brown']
            for i, p in enumerate(points):
                draw.ellipse([p[0]-p_wdth, p[1]-p_wdth,
                              p[0]+p_wdth, p[1]+p_wdth], fill=colors[i])

        return img_copy

    #
    def crop_face(self, image, face):
        box_points, f_center = face['box_points'], face['face_center']

        max_sz = max(
            abs(box_points[0] - f_center[0]),
            abs(box_points[1] - f_center[1]),
            abs(box_points[2] - f_center[0]),
            abs(box_points[3] - f_center[1]),
        )

        new_bbox = (f_center[0] - max_sz, f_center[1] - max_sz,
                    f_center[0] + max_sz, f_center[1] + max_sz)
        
        return image.crop(new_bbox)

    #
    def align_face(self, face, face_info):
        left_eye, right_eye = face_info['key_points'][:2]
        dist = lambda p1, p2: np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

        #print(left_eye, right_eye)

        #
        c = dist(left_eye, right_eye)

        # finding rotation direction
        if left_eye[1] > right_eye[1]:
            point_3rd = (right_eye[0], left_eye[1])
            a = dist(left_eye, point_3rd)
            direction = -1
            cos_ang = a / c
        else:
            point_3rd = (left_eye[0], right_eye[1])
            b = dist(right_eye, point_3rd)
            direction = 1
            cos_ang = b / c 
        
        #cos_a = (b*b + c*c - a*a)/(2*b*c)
        angle = ((np.arccos(cos_ang) * 180) / math.pi)

        #print(angle)

        #
        return face.rotate(angle*direction)

    #
    def get_faces_from_image(self, image):
        # detect faces
        faces_info = self.detect_faces(image)

        if len(faces_info) == 0:
            return None, None
        
        # crop faces
        croped_faces = [self.crop_face(image, info) for info in faces_info]

        # center faces
        centered_faces = [self.align_face(face, info) for face, info in zip(croped_faces, faces_info)]
        
        # resize faces
        resized_faces = [Image.fromarray(cv2.resize(np.array(face), (IMAGE_SIZE,IMAGE_SIZE))) for face in centered_faces]
        
        return resized_faces, faces_info


In [5]:
images_table = pd.read_csv(IMAGES_INFO_TABLE, sep=';')
inference = ImageProcessing()

In [6]:
bad_images = []

for i in tqdm(range(images_table.shape[0])):
    cur_image_path = f"../{images_table['relative_path'][i]}/{images_table['images_name'][i]}"
    cur_image = Image.open(cur_image_path)

    croped_face, _ = inference.get_faces_from_image(cur_image)

    if croped_face is None:
        bad_images.append(i)
        continue

    one_face = croped_face[0]
    one_face.save(f"../{CROPED_IMAGES_DIR}/{images_table['images_name'][i]}")

  0%|          | 229/50000 [00:03<10:11, 81.35it/s]/tmp/ipykernel_40950/2985626437.py:92: RuntimeWarning: invalid value encountered in scalar divide
  cos_ang = b / c
100%|██████████| 50000/50000 [10:56<00:00, 76.15it/s]


In [7]:
print("bad images amount:", len(bad_images))

bad images amount: 47


In [8]:
print(images_table.shape[0])
images_table = images_table.drop(index=bad_images).reset_index(drop=True)
print(images_table.shape[0])

50000
49953


In [9]:
images_table['croped_path'] = 'data/croped_images'

In [11]:
images_table.to_csv(IMAGES_INFO_TABLE, sep=';', index=False)